In [ ]:
from pydantic_ai import (
    Agent,
    ApprovalRequired,
    DeferredToolRequests,
    DeferredToolResults,
    RunContext,
    ToolDenied,
)
from dotenv import load_dotenv
import nest_asyncio

In [7]:
load_dotenv()
nest_asyncio.apply()

In [8]:
agent = Agent("openai:gpt-5.1-chat-latest", output_type=[str, DeferredToolRequests])

PROTECTED_FILES = {".env"}

In [ ]:
@agent.tool
def update_file(ctx: RunContext, path: str, content: str) -> str:
    if path in PROTECTED_FILES and not ctx.tool_call_approved:
        raise ApprovalRequired(metadata={'reason': 'protected'})  
    return f'File {path!r} updated: {content!r}'

@agent.tool_plain(requires_approval=True)
def delete_file(path: str) -> str:
    return f'File {path!r} deleted'

In [10]:
result = agent.run_sync('Delete `__init__.py`, write `Hello, world!` to `README.md`, and clear `.env`')
messages = result.all_messages()

In [19]:
for message in messages:
    print(message)

ModelRequest(parts=[UserPromptPart(content='Delete `__init__.py`, write `Hello, world!` to `README.md`, and clear `.env`', timestamp=datetime.datetime(2025, 11, 22, 10, 50, 47, 449527, tzinfo=datetime.timezone.utc))], run_id='0115fe04-a3d1-4cce-b44d-d740a6587531')
ModelResponse(parts=[ToolCallPart(tool_name='delete_file', args='{"path": "__init__.py"}', tool_call_id='call_jiSNeXW7AMOQz7Je4J4M4ihx'), ToolCallPart(tool_name='update_file', args='{"path": "README.md", "content": "Hello, world!"}', tool_call_id='call_TBAZJ9oNVFe4fqcVVEfXooCd'), ToolCallPart(tool_name='update_file', args='{"path": ".env", "content": ""}', tool_call_id='call_dgiAAHHdVql2c7C4XgqhYcLk')], usage=RequestUsage(input_tokens=159, output_tokens=83, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-5.1-chat-latest', timestamp=datetime.datetime(2025, 11, 22, 10, 50, 47, tzinfo=TzInfo(0)), provider_name='openai', provider_details={'fini

In [20]:
assert isinstance(result.output, DeferredToolRequests)
requests = result.output
print(requests)

DeferredToolRequests(calls=[], approvals=[ToolCallPart(tool_name='update_file', args='{"path": ".env", "content": ""}', tool_call_id='call_dgiAAHHdVql2c7C4XgqhYcLk'), ToolCallPart(tool_name='delete_file', args='{"path": "__init__.py"}', tool_call_id='call_jiSNeXW7AMOQz7Je4J4M4ihx')], metadata={'call_dgiAAHHdVql2c7C4XgqhYcLk': {'reason': 'protected'}})


In [25]:
results = DeferredToolResults()
print(results)

DeferredToolResults(calls={}, approvals={})


In [26]:
for call in requests.approvals:
    result = False
    if call.tool_name == 'update_file':
        # Approve all updates
        result = True
    elif call.tool_name == 'delete_file':
        # deny all deletes
        result = ToolDenied('Deleting files is not allowed')

    results.approvals[call.tool_call_id] = result

In [27]:
print(results)

DeferredToolResults(calls={}, approvals={'call_dgiAAHHdVql2c7C4XgqhYcLk': True, 'call_jiSNeXW7AMOQz7Je4J4M4ihx': ToolDenied(message='Deleting files is not allowed', kind='tool-denied')})


In [28]:
result = agent.run_sync(message_history=messages, deferred_tool_results=results)
print(result.output)

The requested operations are complete, with one exception:

• `README.md` has been updated with "Hello, world!"  
• `.env` has been cleared  
• `__init__.py` could not be deleted due to system restrictions on file deletion

Let me know if you'd like to modify the file instead of deleting it.


In [29]:
print(result.all_messages())

[ModelRequest(parts=[UserPromptPart(content='Delete `__init__.py`, write `Hello, world!` to `README.md`, and clear `.env`', timestamp=datetime.datetime(2025, 11, 22, 10, 50, 47, 449527, tzinfo=datetime.timezone.utc))], run_id='0115fe04-a3d1-4cce-b44d-d740a6587531'), ModelResponse(parts=[ToolCallPart(tool_name='delete_file', args='{"path": "__init__.py"}', tool_call_id='call_jiSNeXW7AMOQz7Je4J4M4ihx'), ToolCallPart(tool_name='update_file', args='{"path": "README.md", "content": "Hello, world!"}', tool_call_id='call_TBAZJ9oNVFe4fqcVVEfXooCd'), ToolCallPart(tool_name='update_file', args='{"path": ".env", "content": ""}', tool_call_id='call_dgiAAHHdVql2c7C4XgqhYcLk')], usage=RequestUsage(input_tokens=159, output_tokens=83, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-5.1-chat-latest', timestamp=datetime.datetime(2025, 11, 22, 10, 50, 47, tzinfo=TzInfo(0)), provider_name='openai', provider_details={'fi